In [50]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import itertools
import time
import random
from sklearn.preprocessing import StandardScaler

import os
import sys
sys.path.append('../src')

from math_utils import *
from model import *
from utils import *

In [51]:
DATA_PATH = '../Concrete_Data.xls' # link dataset: https://archive.ics.uci.edu/dataset/165/concrete+compressive+strength
SEED = 42
df_data = pd.read_excel(DATA_PATH).sample(frac=1, random_state=SEED)

In [52]:
df_data

,Cement (component 1)(kg in a m^3 mixture),Blast Furnace Slag (component 2)(kg in a m^3 mixture),Fly Ash (component 3)(kg in a m^3 mixture),Water (component 4)(kg in a m^3 mixture),Superplasticizer (component 5)(kg in a m^3 mixture),Coarse Aggregate (component 6)(kg in a m^3 mixture),Fine Aggregate (component 7)(kg in a m^3 mixture),Age (day),"Concrete compressive strength(MPa, megapascals)"
31,266.00,114.0,0.00,228.00,0.00,932.0,670.00,365,52.908320
109,362.60,189.0,0.00,164.90,11.60,944.7,755.80,7,55.895819
136,389.90,189.0,0.00,145.90,22.00,944.7,755.80,28,74.497882
88,362.60,189.0,0.00,164.90,11.60,944.7,755.80,3,35.301171
918,145.00,0.0,179.00,202.00,8.00,824.0,869.00,28,10.535193
...,...,...,...,...,...,...,...,...,...
87,286.30,200.9,0.00,144.70,11.20,1004.6,803.70,3,24.400556
330,246.83,0.0,125.08,143.30,11.99,1086.8,800.89,14,42.216615
466,190.34,0.0,125.18,166.61,9.88,1079.0,798.90,100,33.563692
121,475.00,118.8,0.00,181.10,8.90,852.1,781.50,28,68.299493


In [53]:
scaler = StandardScaler()
data = df_data.to_numpy() #converting from excel to numpy array(meanwhile shuffling)

data = scaler.fit_transform(data) #scaling the data

dev_len, test_len = 0.8*(len(data)), 0.2*(len(data)) # 80% for dev and 20% for test
dev_set, test_set = data[:int(dev_len)], data[int(dev_len):]

train_len, val_len = 0.8*(len(dev_set)), 0.2*(len(dev_set)) # 80% for train and 20% for validation
train_set, val_set = dev_set[:int(train_len)], dev_set[int(train_len):]

In [54]:
len(train_set), len(val_set), len(test_set) # 80% for train, 10% for validation and 10% for test

(659, 165, 206)

## Model Selection

In [55]:
def grid_search(config_dict:dict, train_set:np.ndarray, val_set:np.ndarray, config_trials:int=5, metric:callable=mse) -> tuple:
    """
    Perform grid search over the given configuration dictionary.

    Parameters
    ----------
    config_dict : dict
        Dictionary containing the configurations to be tested.
    config_trials : int
        Number of trials for each configuration.

    Returns
    -------
    config_dict : dataframe
        all configurations and their mean and variance of MSE
    """

    metric_name = str(metric.__name__) if hasattr(metric, '__name__') else str(metric)

    df_dict = {'hidden_size': [], 'activation': [], 'init_method': [], 'init_params': [], 'tikhonov': [],
               f'train_{metric_name}': [], f'train_{metric_name}_var': [], f'val_{metric_name}': [], f'val_{metric_name}_var': []}
    # create all possible combination of the configurations
    keys, values = zip(*config_dict.items())
    configs = [dict(zip(keys, v)) for v in itertools.product(*values)]
    for config in configs:
        df_dict['hidden_size'].append(config['hidden_size'])
        df_dict['activation'].append(str(config['activation'].__name__))
        df_dict['init_method'].append(config['init_method'])
        df_dict['init_params'].append(config['init_params'])
        df_dict['tikhonov'].append(config['tikhonov'])

        val_values = []
        train_values = []
        for _ in range(config_trials):
            model = ELM(input_size=config['input_size'], hidden_size=config['hidden_size'], hidden_activation=config['activation'], 
                        init_method=config['init_method'], init_params=config['init_params'])
            
            model.fit(train_set[:, :-1], train_set[:, -1].reshape(-1, 1), config['tikhonov'])

            pred = model.predict(train_set[:, :-1])
            train_values.append(metric(train_set[:, -1].reshape(-1, 1), pred))

            pred = model.predict(val_set[:, :-1])
            val_values.append(metric(val_set[:, -1].reshape(-1, 1), pred))

        df_dict[f'train_{metric_name}'].append(np.mean(train_values))
        df_dict[f'train_{metric_name}_var'].append(np.var(train_values))
        df_dict[f'val_{metric_name}'].append(np.mean(val_values))
        df_dict[f'val_{metric_name}_var'].append(np.var(val_values))

    
    config_df = pd.DataFrame(df_dict)
    return config_df


In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

config_dict = {
    'input_size': [8],
    'hidden_size': [32, 64, 128, 256, 512],
    'activation': [np.tanh, sigmoid],
    'init_method': ['uniform'],
    'init_params': [(-1, 1), (-0.7, 0.7), (-0.4, 0.4)],
    'tikhonov': [1e-6, 1e-4, 1e-8]
}

results_uniform = grid_search(config_dict, train_set, val_set, config_trials=3, metric=mse)

In [ ]:
config_dict = {
    'input_size': [8],
    'hidden_size': [32, 64, 128, 256, 512],
    'activation': [np.tanh, sigmoid],
    'init_method': ['normal'],
    'init_params': [(0, 1)],
    'tikhonov': [1e-6, 1e-4, 1e-8]
}

results_normal = grid_search(config_dict, train_set, val_set, config_trials=3, metric=mse)

In [62]:
all_results = pd.concat([results_uniform, results_normal], axis=0)
all_results = all_results.sort_values(by=f'val_{mse.__name__}', ascending=True)
all_results

,hidden_size,activation,init_method,init_params,tikhonov,train_mse,train_mse_var,val_mse,val_mse_var
17,128,sigmoid,normal,"(0, 1)",1.000000e-08,0.144863,1.511558e-05,0.184058,0.000495
51,128,sigmoid,uniform,"(-0.4, 0.4)",1.000000e-06,0.138984,1.237126e-05,0.190340,0.000400
46,128,sigmoid,uniform,"(-1, 1)",1.000000e-04,0.142246,1.163385e-05,0.199224,0.000151
15,128,sigmoid,normal,"(0, 1)",1.000000e-06,0.148555,4.283649e-05,0.202389,0.000084
50,128,sigmoid,uniform,"(-0.7, 0.7)",1.000000e-08,0.141190,1.521077e-05,0.206870,0.000083
...,...,...,...,...,...,...,...,...,...
86,512,sigmoid,uniform,"(-0.7, 0.7)",1.000000e-08,0.010536,1.287558e-08,28.169527,4.300162
80,512,tanh,uniform,"(-0.4, 0.4)",1.000000e-08,0.010765,1.696418e-07,36.272643,224.538817
84,512,sigmoid,uniform,"(-0.7, 0.7)",1.000000e-06,0.010693,4.383052e-07,41.641238,253.472538
89,512,sigmoid,uniform,"(-0.4, 0.4)",1.000000e-08,0.010360,6.370816e-08,53.758232,819.386980


## Test Assessment

In [64]:
hidden_size = all_results.iloc[0]['hidden_size']
activation = sigmoid if all_results.iloc[0]['activation'] == 'sigmoid' else np.tanh
init_method = all_results.iloc[0]['init_method']
init_params = all_results.iloc[0]['init_params']
tikhonov = all_results.iloc[0]['tikhonov']

model = ELM(input_size=8, hidden_size=hidden_size, hidden_activation=activation, init_method=init_method, init_params=init_params)
model.fit(dev_set[:, :-1], dev_set[:, -1].reshape(-1, 1), tikhonov)

pred = model.predict(test_set[:, :-1])
test_mse = mse(test_set[:, -1].reshape(-1, 1), pred)
print(f'Test MSE: {test_mse}')

Test MSE: 0.22130252652515012
